In [3]:
if(!require(ggplot2)){install.packages('ggplot2');library(ggplot2)}else{library(ggplot2)}
if(!require(stringr)){install.packages('stringr');library(stringr)}else{library(stringr)}
if(!require(plotly)){install.packages('plotly');library(plotly)}else{library(plotly)}
if(!require(utils)){install.packages('utils');library(utils)}else{library(utils)}
if(!require(signal)){install.packages('signal');library(signal)}else{library(signal)}
if(!require(reshape2)){install.packages('reshape2');library(reshape2)}else{library(reshape2)}
if(!require(scales)){install.packages('scales');library(scales)}else{library(scales)}

In [4]:
lee_firmas=function(directorio_firmas,Tratamiento,Niveles,Replicas,fecha_ini,extension,decimal,separacion,delte_line,lon_ini,lon_fin){
  Tratamiento=as.character(Tratamiento)
  Niveles=as.character(Niveles)
  if(exists("Replicas")){
    Replicas=as.character(Replicas) 
  }
  setwd(directorio_firmas)
  folders=list.dirs(getwd(),full.names=T)
  for(i in 1:length(Tratamiento)){
    f=Tratamiento[i]
    foldersT=str_subset(folders,paste0("/",f,"/"))
    matrix.fir=lapply(Niveles,function(x){
      # x=Niveles[1]
      q=str_subset(foldersT,paste0("/",x,"/"))
      e=list()
      suppressWarnings (for (j in 1:length(Replicas)) {
        # j=1
        r=str_subset(q,paste0("/",Replicas[j]))
        if((str_sub(r,min(nchar(r)),max(nchar(r)))==Replicas[j])==F){
          r
        }else{
          r=r[str_sub(r,min(nchar(r)),max(nchar(r)))==Replicas[j]]
        }
        fir_1=str_subset(list.files(r,full.names = T),extension)
        longi=as.data.frame(read.delim(fir_1[1],header = F,sep = separacion,dec = decimal,skip =delte_line))$V1
        lon=c(which(longi==lon_ini):which(longi==lon_fin))
        fir_1=t(sapply(fir_1, function(t){as.data.frame(read.delim(t,header = F,sep = separacion,dec = decimal,skip =delte_line))[lon,2]}))
        fechas=na.omit(unique(str_locate(row.names(fir_1),fecha_ini)))
        fechas=str_sub(row.names(fir_1),fechas[1,1],fechas[1,2])
        len=max(unlist(str_locate_all(row.names(fir_1)[1],"/")))
        id=paste0(fechas,"_",f,"_",x,"_",Replicas[[j]],"_",str_remove(str_sub(row.names(fir_1),len+1,len+30),paste0("-",Replicas[[j]],extension)))
        row.names(fir_1)=id;colnames(fir_1, do.NULL = FALSE);colnames(fir_1)=longi[lon]
        outliers= ifelse(fir_1>=0 & fir_1<=100,0,1);outliers=apply(outliers,1,sum);names(which(outliers>0));fir_1=fir_1[-which(outliers>0),]
        e[[j]]=fir_1
      })
      names(e)=Replicas
      e
    })
    names(matrix.fir)=Niveles
    matrix.fir
    dir=paste0(getwd(),"/",f)
    if(file.exists(dir)){
      dir=paste0(dir,"/Matrices_Firmas")
      if(!file.exists(dir)){
        dir.create(dir) 
      }
    }else{
      dir.create(dir)
      dir=paste0(dir,"/Matrices_Firmas")
      dir.create(dir)
    }
    lapply(Niveles, function(q){
      for (w in 1:length(Replicas)) {
        np=which(names(matrix.fir)==q)
        rp=which(names(matrix.fir[[np]])==Replicas[w])
        
        db=matrix.fir[[np]][[rp]]
        write.csv2(db,
        paste0(dir,"/",f,"_",names(matrix.fir)[np],"_",names(matrix.fir[[np]])[rp],".csv"),
        row.names = TRUE)
      }
    })
    
    assign(paste0(f,".fir"),matrix.fir,envir = .GlobalEnv)
    }
}